In [ ]:
import pandas as pd
import datetime
from IPython.display import display, HTML
import pyperclip

display(HTML("<style>.container { width:100% !important; }</style>"))

#грузим и обрабатываем данные
df = pd.read_csv('corona.csv',sep=';',encoding='utf8')
df['dt'] = pd.to_datetime(df['Дата'],format='%d.%m.%Y')
df = df.drop(columns=['Дата'])
df.rename(columns = {
    'Регион':'region',
    'Заражений':'infected_total',
    'Выздоровлений':'healed_total',
    'Смертей':'deaths',
    'Смертей за день':'deaths_per_day',
    'Заражений за день':'infected_per_day',
    'Выздоровлений за день':'healed_per_day',
},inplace=True)

dft = pd.read_excel('regions.xlsx') #словарь с координатами плиток

# объединяем со справочником
df = df.merge(dft,how='left',left_on='region',right_on='corona_region')

# датасет для плитки (с координатами положения регионов)
df_plitka = df[['region','row','column']].drop_duplicates()

# df.to_csv('dataset_corona_with_regions.csv',index=False) #сохраняю для загрузки в datalens

### Календарь

In [ ]:
month = 11

width_big = '370px' #ширина внешней рамки
width = '360px' #ширина плиток
cell_height = '25px'
grid_height = '35px'


month_name = datetime.date(1900,month,1).strftime("%B")
weeks = 6
days = 7



text = ''
text+=f"""\n'<div style="border:1px solid #d4d4d4;border-radius:5px;width: {width_big};height: 370px;">'+"""
text+=f"""\n'<div style="border:1px solid #000;border-radius:5px;background-color:#fff;margin: 4px;padding-left: 10px;width: {width};height: {cell_height};border-color: grey;display: flex; align-items: center;">'+
'<span style="font-size:14px;">{month_name} '+str(year([p_report_date]))+'</span></div>'+ """
for week in range(weeks):
    text+=f"""\n   '<div style="display: grid;grid-template-columns: repeat({days}, 1fr);grid-gap: 2px;border: 0px solid #696989;width: {width};height: {grid_height};padding: 1px;border: 0px solid 696989;margin-left: 4px;margin-right: 4px;margin-bottom:19px;">'+"""
    for day in range(1,days+1):
        color_string = f'''\n if(sum_if([healed_per_day],{day+week*7}-DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))+1 = day([dt]) and month([dt])={month} and year([dt])=year([p_report_date]))>sum_if([infected_per_day],{day+week*7}-DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))+1 = day([dt]) and month([dt])={month} and year([dt])=year([p_report_date])),'#006600','#990000')'''
        
        text+=f"""\n    if(DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))-1 < {day+week*7} and day(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month})-1)+DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))-1 >= {day+week*7},
'<div style="border:1px solid #000;border-radius:5px;background-color: {'#f5f5f5' if day <= 5 else '#fff0f5'}; border-color: grey; padding: 1px;">'+
'<span style="font-size:10px;color: #bababa;">'+
str({day+week*7}-DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))+1)
+'</span></p>'
+'<span style="font-size:14px; color: '+{color_string}+';display: table;margin: 0 auto;">'+
str(if(DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))-1 < {day+week*7} and day(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month})-1)+DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))-1 >= {day+week*7},
str(round(sum_if([healed_per_day],{day+week*7}-DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))+1 = day([dt]) and month([dt])={month} and year([dt])=year([p_report_date]))
/
sum_if([infected_per_day],{day+week*7}-DAYOFWEEK(DATEADD(DATETRUNC([p_report_date],'year'),'month',{month-1}))+1 = day([dt]) and month([dt])={month} and year([dt])=year([p_report_date])),2))
,''))+'</span></div>',
'<div style="border:0px solid #000;border-radius:5px;background-color:fff; border-color: grey; padding: 0px;height: {cell_height};"><span></span></div>')+
"""
    text+="'</div>'+"
text+= "'</div>'"

pyperclip.copy(text)
print(len(text))

70529


### Плитки

In [13]:
df_plitka

,region,row,column
0,Чукотский АО,2,17
1,Бурятия,5,15
2,Краснодарский край,10,5
3,Адыгея,11,5
4,Челябинская обл.,6,11
...,...,...,...
139,Саха (Якутия),3,16
142,Тамбовская обл.,7,6
149,Тюменская обл.,4,12
161,Рязанская обл.,6,6


In [ ]:
row_num = 12 #задается для 1 графика (который отвечает за строку плиточной карты)
#--параметры для настройки-----------------------
grid_gap = '8px' #расстояние между плитками слева-справа
height = '55px' #высота 1 строки
line_width = '1580px' #ширина всей строки

cell_width = '80px' #ширина 1 клетки
border_cell_width = '1px' #ширина границы
border_cell_color = '#000' #цвет границы
border_radius = '5px' #скругление углов
font_size_name = '8px' #размер названия региона
font_size_val = '16px' #размер шрифта значения
#------------------------------------------------


row_df = df_plitka[df_plitka['row']==row_num] #оставляем только нужную строку

def shorten_name(x):
    '''функция для сокращения кол-ва симоволов в подписи плитки'''
    x = x.replace('Калининградская','Калинингр.')
    x = x.replace('Ленинградская','Ленингр.')
    x = x.replace('Ямало-Ненецкий','Ям.-Нен.')
    x = x.replace('Красноярский','Красноярск.')
    x = x.replace('Нижегородская','Нижегородск.')
    x = x.replace('Забайкальский','Забайкальск.')
    x = x.replace('Краснодарский','Краснод.')
    x = x.replace('Ставропольский','Ставропол.')
    x = x.replace('Карачаево-Черкессия','Карач.-Черк.')
    x = x.replace('Кабардино-Балкария','Кабард.-Балк.')
    x = x.replace('Новосибирская','Новосиб.')
    return x

text = """'<div style="background-color: #fff;">'+\n"""
text+=f"""'<div style="display:grid;grid-template-columns:repeat({df_plitka.column.max()},1fr);grid-gap:{grid_gap};width:{line_width};height:{height};padding:0px;border:0px;">'+"""
for col in range(1,df_plitka.column.max()):
    is_empty = len(row_df[row_df['column']==col]['region'])==0                 #смотрим клетка содержит регион или нет
    name = '' if is_empty else row_df[row_df['column']==col]['region'].iloc[0] #достаем название региона
    background_color = '#fff' if is_empty else '#ededed'                       #цвет фона
    border_cell = '0px' if is_empty else border_cell_width
    number = "''" if is_empty else f"""str(round(sum_if([healed_per_day],[region]='{name}')/sum_if([infected_per_day],[region]='{name}'),2))""" #формула для отношения щаболевших к вылечившимся
    font_val_color = f"""if(round(sum_if([healed_per_day],[region]='{name}')/sum_if([infected_per_day],[region]='{name}'),2)<1,'color: #990000;">','color: #006600;">')"""
    
    
    text+=f"""\n '<div style ="border: {border_cell} solid {border_cell_color};border-radius: {border_radius};background-color: {background_color};height: {height};width: {cell_width};padding: 2px;font-size: {font_size_name}">{shorten_name(name)}'+"""
    text+=f"""'<p style ="font-size: {font_size_val}; '+{font_val_color}+{number}+'</p></div>'+"""
text+="\n'</div></div>'"

pyperclip.copy(text)
print(len(text))

5714
